# Exercises for 2021-03-04

In [ ]:
using LinearAlgebra
using Plots
using Random

## Clustering by interpolative decomposition

Consider a data set consisting of $k$ clusters in $n$-dimensional space with $n > k$. These clusters might conventionally be recovered by k-means iteration, but if they are sufficiently well separated, they can also be computed via an interpolative decomposition; if $A \in \mathbb{R}^{n \times m}$ is the data set (consisting of $m$ points), then we have the approximate rank $k$ decomposition
$$A \approx CT$$
where $C \in \mathbb{R}^{n \times k}$ is a subset of representative columns of $A$ and the location of the largest element in each column of T indicates the cluster to which the corresponding column of A belongs.

In [ ]:
## Noise level
σ = 0.2

## Generate cluster data
centers = randn(5,3)
e10     = ones(1,10)
A       = zeros(5,30)
A[:,1:10]  .= centers[:,1]
A[:,11:20] .= centers[:,2]
A[:,21:30] .= centers[:,3]
An = A + σ*randn(size(A))

## Compute the pivoted QR decomposition of the noisy A matrix
Q, R, p = qr(An, Val(true))

## Reconstruct C and check vs QR decomp
C = An[:,p[1:3]]
Q1 = Q[:,1:3]
R1 = R[1:3,1:3]
println("norm(C-Q1*R1) = $(norm(C-Q1*R1))")

As we are going to be doing some 2D plots of the clusters in different coordinate systems, it will be convenient to add a custom plotter for this problem.

In [ ]:
function scatter_clusters(x, y)
    scatter(x[1:10], y[1:10], color=:red)
    scatter!(x[11:20], y[11:20], color=:blue)
    scatter!(x[21:30], y[21:30], color=:black)
end

In [ ]:
## TODO: Evaluate T

TODO: Run the plot command below.  What do you observe?  Why?

In [ ]:
## 2D scatter plot
scatter_clusters(T[1,:], T[2,:])

In [ ]:
## TODO: Determine cluster labels from max elt in each column
#cluster_ids = [SOMETHING for i = 1:30]

## PCA plotting

We now consider two flavors of plotting a 2D projection of the data: one taking into account the class labels, the other one not.  Before getting started, though, let's center the data.

In [ ]:
means = [sum(An[i,:]) for i=1:5]./30
Cn = An.-means

In [ ]:
# Project the centered data in the direction of the two dominant singular vectors and plot
U,S,Vt = svd(Cn)
Qsvd = U[:,1:2]
scatter_clusters(S[1]*Vt[:,1], S[2]*Vt[:,2])

TODO: The code above is equivalent to plotting `U[:,1:2]'*Cn`.  Why?

TODO: Print the singular values.  You should find that two of them are significantly larger than the rest.  Can you explain why geometrically?

## Multiclass LDA

Now we consider the multi-class LDA approach, where the coordinates are determined by the eigenvectors for the problem
$$
  \Sigma_b w = \lambda \Sigma w
$$
where $\Sigma_b$ is the between-class scatter and $\Sigma$ is the overall covariance.

In [ ]:
# Compute class means
μ1 = sum(Cn[:,j] for j = 1:10)/10
μ2 = sum(Cn[:,j] for j = 11:20)/10
μ3 = sum(Cn[:,j] for j = 21:30)/10
μ = (μ1 + μ2 + μ3)/3

# Between-class scatter matrix
Σb = ( (μ1-μ)*(μ1-μ)' + (μ2-μ)*(μ2-μ)' + (μ3-μ)*(μ3-μ)' )/3

# Covariance (taking normalization into account)
Σ = Cn*Cn'

# Compute the space associated with the largest vectors
λs, Ws = eigen(Σb, Σ)
Qlda, Rlda = qr(Ws[:,4:5])
Qlda = Array(Qlda)
Qlda = Qlda[:,1:2]

# Project the centered data onto this space
xy_lda = Qlda'*Cn

# And plot
scatter_clusters(xy_lda[1,:], xy_lda[2,:])

## Compare and contrast

The objective for the SVD space is to minimize
$$
  \phi_{SVD}(Q) = \frac{\mathrm{tr}(Q^T Q)}{\mathrm{tr}(Q^T \Sigma Q)}
$$
while the objective for the LDA space is to minimize
$$
  \phi_{LDA}(Q) = \frac{\mathrm{tr}(Q^T \Sigma_b Q)}{\mathrm{tr}(Q^T \Sigma Q)}.
$$
This general trace-minimization framework is described in rather more detail in the paper by Kokiopoulou, Chen, and Saad linked from the class web page.

TODO: Numerically verify that the SVD objective is smaller on the SVD space than the LDA space, and vice-versa.

TODO: The clusters appear fairly well-defined for the given noise level ($\sigma = 0.2$).  Try increasing $\sigma$ (say to 0.4 or 0.5) and re-running the experiments.  What do you observe?